# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [13]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
pd.read_sql("""
            SELECT *
            FROM offices           
            LIMIT 10;
            """, conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [14]:
# Your code here
q = """
SELECT lastName, firstName
FROM employees
JOIN offices
     ON employees.officeCode = offices.officeCode
     WHERE offices.city = 'Boston';
    """
pd.read_sql(q, conn)

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [15]:
# Your code here
q = """
SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS totalOffice_employees
FROM offices o
LEFT JOIN employees e
    ON o.officeCode = e.officeCode
GROUP BY o.officeCode, o.city;
"""
pd.read_sql(q, conn)

,officeCode,city,totalOffice_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,0


## Write 3 questions of your own and answer them

In [24]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
q = """
SELECT e.officeCode, COUNT(c.customerNumber) AS no_ofCustomerPerOffice
FROM employees e
JOIN customers c
ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.officeCode;
"""
pd.read_sql(q, conn)

,officeCode,no_ofCustomerPerOffice
0,1,12
1,2,12
2,3,15
3,4,29
4,5,5
5,6,10
6,7,17


In [36]:
"""
Which employees generate the highest revenue?
"""

# Your code here
q = """
SELECT 
e.employeeNumber,
e.firstName,
e.lastName,
ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS total_revenue
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.firstName, e.lastName
ORDER BY total_revenue DESC;
"""
pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,total_revenue
0,1370,Gerard,Hernandez,1258577.81
1,1165,Leslie,Jennings,1081530.54
2,1401,Pamela,Castillo,868220.55
3,1501,Larry,Bott,732096.79
4,1504,Barry,Jones,704853.91
5,1323,George,Vanauf,669377.05
6,1612,Peter,Marsh,584593.76
7,1337,Loui,Bondur,569485.75
8,1611,Andy,Fixter,562582.59
9,1216,Steve,Patterson,505875.42


In [37]:
"""
Which products are sold most frequently?Products by Units sold
"""

# Your code here
q = """
SELECT 
p.productCode,
p.productName,
SUM(od.quantityOrdered) AS total_units_sold
FROM products p
JOIN orderdetails od
    ON p.productCode = od.productCode
GROUP BY 
    p.productCode, p.productName
ORDER BY total_units_sold DESC;
"""
pd.read_sql(q, conn)


,productCode,productName,total_units_sold
0,S18_3232,1992 Ferrari 360 Spider red,1808
1,S18_1342,1937 Lincoln Berline,1111
2,S700_4002,American Airlines: MD-11S,1085
3,S18_3856,1941 Chevrolet Special Deluxe Cabriolet,1076
4,S50_1341,1930 Buick Marquette Phaeton,1074
...,...,...,...
104,S18_2870,1999 Indy 500 Monte Carlo SS,855
105,S18_2248,1911 Ford Town Car,832
106,S24_3969,1936 Mercedes Benz 500k Roadster,824
107,S24_1046,1970 Chevy Chevelle SS 454,803


In [38]:
"""
What is the revenue vs units sold?
"""

# Your code here
q = """
SELECT 
p.productCode,
p.productName,
SUM(od.quantityOrdered) AS units_sold,
ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS revenue,
    ROUND(
        SUM(od.quantityOrdered * od.priceEach) 
        / SUM(od.quantityOrdered), 
        2
    ) AS revenue_per_unit
FROM products p
JOIN orderdetails od
    ON p.productCode = od.productCode
GROUP BY 
    p.productCode, p.productName
ORDER BY revenue DESC;
"""
pd.read_sql(q, conn)

,productCode,productName,units_sold,revenue,revenue_per_unit
0,S18_3232,1992 Ferrari 360 Spider red,1808,276839.98,153.12
1,S12_1108,2001 Ferrari Enzo,1019,190755.86,187.20
2,S10_1949,1952 Alpine Renault 1300,961,190017.96,197.73
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,985,170686.00,173.29
4,S12_1099,1968 Ford Mustang,933,161531.48,173.13
...,...,...,...,...,...
104,S32_2206,1982 Ducati 996 R,906,33268.76,36.72
105,S24_2840,1958 Chevy Corvette Limited Edition,983,31627.96,32.17
106,S24_2972,1982 Lamborghini Diablo,912,30972.87,33.96
107,S24_3969,1936 Mercedes Benz 500k Roadster,824,29763.39,36.12


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [28]:
# Your code here
q = """
SELECT 
e.employeeNumber,
e.firstName,
e.lastName,
p.productName
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
JOIN products p
    ON od.productCode = p.productCode
ORDER BY  e.employeeNumber,e.lastName, e.firstName, p.productName;
"""
pd.read_sql(q, conn)


,employeeNumber,firstName,lastName,productName
0,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
1,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
2,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
3,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
4,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
...,...,...,...,...
2991,1702,Martin,Gerard,The Titanic
2992,1702,Martin,Gerard,The Titanic
2993,1702,Martin,Gerard,The Titanic
2994,1702,Martin,Gerard,The USS Constitution Ship


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [29]:
# Your code here
q= """
SELECT 
e.employeeNumber,
e.firstName,
e.lastName,
SUM(od.quantityOrdered) AS total_products_sold
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.firstName, e.lastName
ORDER BY total_products_sold DESC;
"""
pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,total_products_sold
0,1370,Gerard,Hernandez,14231
1,1165,Leslie,Jennings,11854
2,1401,Pamela,Castillo,9290
3,1501,Larry,Bott,8205
4,1504,Barry,Jones,7486
5,1323,George,Vanauf,7423
6,1612,Peter,Marsh,6632
7,1611,Andy,Fixter,6246
8,1337,Loui,Bondur,6186
9,1216,Steve,Patterson,5561


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [35]:
# Your code here
q = """
SELECT 
e.employeeNumber,
e.firstName,
e.lastName,
COUNT(DISTINCT od.productCode) AS distinct_products_sold
FROM employees e
JOIN customers c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o
    ON c.customerNumber = o.customerNumber
JOIN orderdetails od
    ON o.orderNumber = od.orderNumber
GROUP BY 
    e.employeeNumber, e.firstName, e.lastName
HAVING COUNT(DISTINCT od.productCode) > 200
ORDER BY distinct_products_sold DESC;
"""
pd.read_sql(q, conn)

,employeeNumber,firstName,lastName,distinct_products_sold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!